In [1]:
!rm -rf InkubaLM-Challenge
!git clone https://github.com/melissafasol/InkubaLM-Challenge.git
%cd InkubaLM-Challenge


Cloning into 'InkubaLM-Challenge'...
remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 443 (delta 20), reused 18 (delta 9), pack-reused 409 (from 1)
Receiving objects: 100% (443/443), 1.42 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (294/294), done.
/content/InkubaLM-Challenge


In [36]:
%cd InkubaLM-Challenge
!git pull


[Errno 2] No such file or directory: 'InkubaLM-Challenge'
/content/InkubaLM-Challenge
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 330 bytes | 330.00 KiB/s, done.
From https://github.com/melissafasol/InkubaLM-Challenge
   5113893..2750b41  main       -> origin/main
Updating 5113893..2750b41
Fast-forward
 utils/multitask_v2.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install -U trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install datasets

In [37]:
import sys
sys.path.append("..")  # Add parent directory to the path

import os
from typing import List
from pathlib import Path
import numpy as np

# DO NOT EDIT
# create submission file
import pandas as pd
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
)
from utils import (
    eval,
    model_function,
    multitask,
    experiment_logger,
    supplement,
    multitask_v2
    )

import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, concatenate_datasets, Dataset, Value
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [7]:
from google.colab import drive
drive.mount('/content/drive')
output_path = "/content/drive/MyDrive/InkubaLM-Challenge/Output"
os.makedirs(output_path, exist_ok=True)

Mounted at /content/drive


In [8]:
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

from huggingface_hub import login

try:
    from google.colab import userdata

    # Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
    # vars as appropriate for your system.
    # userdata.get("HF_TOKEN") indicates that the name of the token in the Colab env is HF_TOKEN
    os.environ["hf_token_2"] = userdata.get("hf_token_2")
except:
    os.environ["hf_token_2"] = "----"

login(token=os.environ["hf_token_2"])

token = os.environ["hf_token_2"]
if token == "----":
    print("⚠️ Warning: No Hugging Face token found. Some models may not load.")
else:
    login(token=token)

In [9]:
hf_token_2 = '..' # paste your token here
os.environ["HF_TOKEN"] = hf_token_2

## **Load and Explore Datasets**

In [10]:
print("# Loading datasets")
train_dataset = multitask.load_and_combine_datasets("Train")
test_dataset = multitask.load_and_combine_datasets("Test")

print("\n# Example from training dataset:")
print(train_dataset[0])

print("\n# Example from test dataset:")
print(test_dataset[0])

# Loading datasets


README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/39.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

...

Common Columns: ['langs', 'targets', 'inputs', 'instruction', 'ID']


Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]


# Example from training dataset:
{'ID': 'ID_6aba33a1_sentiment_ dev_hausa', 'langs': 'hausa', 'instruction': 'Za ka iya tantance yanayin wannan rubutu? Bi waɗannan jagororin sharhi: Kyakkyawa: idan rubutu na nuna kyakkyawan tunani, hali, da yanayi. Korau: idan rubutu yana nuna mummunar tunani ko yanayi. Neutral: idan rubutu baya nuna kyakkyawar magana ko mara kyau kai tsaye ko a kaikaice.', 'inputs': '@user @user allah ya tsayyaba yar uwa 🎂 😍', 'targets': 'Kyakkyawa'}

# Example from test dataset:
{'ID': 'ID_f3c74c7b_sentiment_test__hausa', 'langs': 'hausa', 'instruction': "Gano ra'ayin da aka bayyana a cikin wannan rubutu. Bin waɗannan jagororin, kyakkyawa yana na rubutu na nufin kyakkyawan tunani, ɗabi'a, da motsin rai. Korau na nuna rubutu na nufin mummunan tunani ko motsin rai. Tsaka-tsaki na nuna rubutu baya nufin magana mai kyau ko mara kyau kai tsaye ko a kaikaice.", 'inputs': '@user ynxu fha da kanada kudi shikenan duk kayan nan zasu iya zama naka no🧢', 'targets': None}


In [11]:
print("# Converting to DataFrame and extracting task types")
train_df = train_dataset.to_pandas()
train_df['task'] = train_df.ID.apply(multitask.extract_task_from_id)

print("\n# Dataset distribution by task:")
print(train_df.task.value_counts())

# Converting to DataFrame and extracting task types

# Dataset distribution by task:
task
mt           600
sentiment    400
afrixnli     400
Name: count, dtype: int64


In [12]:
train_df.loc[train_df['task'] == 'afrixnli']

,ID,langs,instruction,inputs,targets,task
1000,ID_648d37ff_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?","Bayan nace e, ya ƙare.",0,afrixnli
1001,ID_f96a39cb_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Ba zan damu ba idan ɗaukan nauyin kuɗin da ake...,0,afrixnli
1002,ID_4c5d953b_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?","Alas, hakuweza kuelewa wazi kutokana na kizuiz...",1,afrixnli
1003,ID_99a61e3d_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Nace a'a amma yaci gaba da ja-in-ja.,2,afrixnli
1004,ID_a38aa9d8_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?",Kanuni haziwezi kuamua jinsi ya kutengeneza mk...,1,afrixnli
...,...,...,...,...,...,...
1395,ID_085354e1_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?",Maduka ya habari ya kitaifa hufanya maeneo yet...,1,afrixnli
1396,ID_586e104a_dev_afrixnli_swa,swa,"Is the following question True, False or Neither?",Kurasa zilihusisha wanachama na maafisa wa kaw...,0,afrixnli
1397,ID_b871ea53_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Ban damu ba da abinda labarun ƙasa ke nuni cik...,2,afrixnli
1398,ID_70aae970_dev_afrixnli_hau,hau,"Is the following question True, False or Neither?",Ya zabi ƙin kama hannayen sa saboda anyi musu ...,0,afrixnli


Analyse target sequence lengths (without balancing)

In [13]:
unbalanced_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))

print("\n# The problem: Tasks with shorter outputs (like sentiment) will be underrepresented")
print("in the loss function compared to tasks with longer outputs (like mt).")


# The problem: Tasks with shorter outputs (like sentiment) will be underrepresented
in the loss function compared to tasks with longer outputs (like mt).


In [90]:
print("\n# Applying target length balancing fix")
balanced_df = multitask.balance_target_lengths(train_df)

print("\n# Analyzing target sequence lengths after balancing")
task_stats_after = multitask.analyze_task_lengths(balanced_df)



# Applying target length balancing fix

# Analyzing target sequence lengths after balancing
Target sequence length analysis by task:
Task: sentiment
count    400.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------
Task: mt
count    600.000000
mean      11.801667
std        8.347163
min        2.000000
25%        6.000000
50%       10.000000
75%       15.000000
max       62.000000
Name: targets, dtype: float64
--------------------------------------------------
Task: afrixnli
count    400.0
mean      12.0
std        0.0
min       12.0
25%       12.0
50%       12.0
75%       12.0
max       12.0
Name: targets, dtype: float64
--------------------------------------------------


In [14]:
sentiment_swa_supp = supplement.generate_swahili_supplement()
sentiment_hau_supp = supplement.generate_hausa_supplement()

In [15]:
sent_df_supp = pd.concat([sentiment_swa_supp,sentiment_hau_supp])

In [16]:
sent_df_supp

,ID,task,langs,instruction,inputs,targets
0,ID_afe90ad5_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Hongera kwa kazi nzuri!,Chanya
1,ID_a883ca0f_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Endelea hivyo hivyo!,Chanya
2,ID_b85ddb8d_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Ujumbe huu unatia matumaini.,Chanya
3,ID_e4e49c9f_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Huduma ilikuwa ya kiwango cha juu.,Chanya
4,ID_e7ba44e0_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Huduma ilikuwa ya kiwango cha juu.,Chanya
...,...,...,...,...,...,...
295,ID_aa95fe47_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina zaune a gida ina kallon talabijin.,Tsaka-tsaki
296,ID_f461ffbd_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Za mu je kasuwa gobe da safe.,Tsaka-tsaki
297,ID_8395a3f1_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Ina jiran zuwan motar haya.,Tsaka-tsaki
298,ID_2bc8bcb0_sentiment_dev_hausa,sentiment,hausa,Da fatan za a gano ra'ayin da ke cikin wannan ...,Za mu gana da su bayan sallar magariba.,Tsaka-tsaki


In [17]:
sent_df_supp.head()

,ID,task,langs,instruction,inputs,targets
0,ID_afe90ad5_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Hongera kwa kazi nzuri!,Chanya
1,ID_a883ca0f_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Endelea hivyo hivyo!,Chanya
2,ID_b85ddb8d_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Ujumbe huu unatia matumaini.,Chanya
3,ID_e4e49c9f_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Huduma ilikuwa ya kiwango cha juu.,Chanya
4,ID_e7ba44e0_sentiment_dev_swahili,sentiment,swahili,Changanua mawazo ya matini yanayofuata na uain...,Huduma ilikuwa ya kiwango cha juu.,Chanya


In [339]:
train_df.head()

,ID,task,langs,data_source,instruction,inputs,targets,premise
0,ID_6aba33a1_sentiment_ dev_hausa,sentiment,hausa,afrisenti,Za ka iya tantance yanayin wannan rubutu? Bi w...,@user @user allah ya tsayyaba yar uwa 🎂 😍,Kyakkyawa,
1,ID_ce64d307_sentiment_ dev_hausa,sentiment,hausa,naijasenti,Da fatan za a gano ra'ayin da ke cikin wannan ...,@user intenet a masallachi😭😭😭 wani salo ne na ...,Tsaka-tsaki,
2,ID_dfb02831_sentiment_ dev_swahili,sentiment,swahili,swahili_tweet,Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...,picha mbunge wa kilombero peter lijualikali ak...,Wastani,
3,ID_2efc9515_sentiment_ dev_hausa,sentiment,hausa,afrisenti,Gano ra'ayin da aka bayyana a cikin wannan rub...,@user @user @user @user @user hhh amma rahama ...,Tsaka-tsaki,
4,ID_ad1d9888_sentiment_ dev_swahili,sentiment,swahili,afrisenti,Tafadhali tambua mawazo yaliyoonyeshwa kwenye ...,swali zuri sana nawatafuta wajuzi wa mambo wat...,Wastani,


In [21]:
print("# Loading datasets")
train_dataset = load_and_combine_datasets("Train")
test_dataset = load_and_combine_datasets("Test")

print("\n# Example from training dataset:")
print(train_dataset[0])

print("\n# Example from test dataset:")
print(test_dataset[0])

# Loading datasets
All Columns: ['data_source', 'langs', 'targets', 'inputs', 'instruction', 'task', 'ID', 'premise']


Casting the dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/600 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

All Columns: ['data_source', 'langs', 'targets', 'inputs', 'instruction', 'task', 'ID', 'premise']


Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/300 [00:00<?, ? examples/s]


# Example from training dataset:
{'ID': 'ID_6aba33a1_sentiment_ dev_hausa', 'task': 'sentiment', 'langs': 'hausa', 'data_source': 'afrisenti', 'instruction': 'Za ka iya tantance yanayin wannan rubutu? Bi waɗannan jagororin sharhi: Kyakkyawa: idan rubutu na nuna kyakkyawan tunani, hali, da yanayi. Korau: idan rubutu yana nuna mummunar tunani ko yanayi. Neutral: idan rubutu baya nuna kyakkyawar magana ko mara kyau kai tsaye ko a kaikaice.', 'inputs': '@user @user allah ya tsayyaba yar uwa 🎂 😍', 'targets': 'Kyakkyawa', 'premise': ''}

# Example from test dataset:
{'ID': 'ID_f3c74c7b_sentiment_test__hausa', 'task': 'sentiment', 'langs': 'hausa', 'data_source': 'afrisenti', 'instruction': "Gano ra'ayin da aka bayyana a cikin wannan rubutu. Bin waɗannan jagororin, kyakkyawa yana na rubutu na nufin kyakkyawan tunani, ɗabi'a, da motsin rai. Korau na nuna rubutu na nufin mummunan tunani ko motsin rai. Tsaka-tsaki na nuna rubutu baya nufin magana mai kyau ko mara kyau kai tsaye ko a kaikaice.",

In [22]:
train_df['task'] = train_df.ID.apply(extract_task_from_id)

print("\n# Dataset distribution by task:")
print(train_df.task.value_counts())

# Converting to DataFrame and extracting task types

# Dataset distribution by task:
task
mt           600
sentiment    400
afrixnli     400
Name: count, dtype: int64


In [23]:
required_columns = ['ID', 'task', 'langs', 'instruction', 'inputs', 'targets', 'data_source', 'premise']

for df in [train_df, sent_df_supp]:
    for col in required_columns:
        if col not in df.columns:
            df[col] = ""  # fill missing columns with empty string

# Reorder columns to be consistent
df1 = train_df[required_columns]
df2 = sent_df_supp[required_columns]

# Concatenate without NaNs
combined_df = pd.concat([df1, df2], ignore_index=True)

In [25]:
print("\n# Dataset distribution by task:")
print(combined_df.task.value_counts())


# Dataset distribution by task:
task
sentiment    900
mt           600
afrixnli     400
Name: count, dtype: int64


In [26]:
print("\n# Applying target length balancing fix")
#balanced_df = balance_target_lengths(train_df)
balanced_df = balance_target_lengths(combined_df)


# Applying target length balancing fix


In [27]:
balanced_dataset = Dataset.from_pandas(balanced_df.reset_index(drop=True))

In [28]:
model_name = "lelapa/InkubaLM-0.4B"
# turn off qlora: use_4bit=False
model, tokenizer, bnb_config = setup_model_and_tokenizer(model_name, use_4bit=False)

config.json:   0%|          | 0.00/763 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/991k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [29]:
balanced_dataset

Dataset({
    features: ['ID', 'task', 'langs', 'instruction', 'inputs', 'targets', 'data_source', 'premise'],
    num_rows: 1900
})

In [30]:
balanced_trainer = multitask_v2.setup_trainer(
    model=model,
    dataset=balanced_dataset,
    tokenizer=tokenizer,
    output_dir="./sft_model/balanced_supp"
)
balanced_trainer.train()

Applying formatting function to train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1900 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Step,Training Loss
10,1.463100
20,1.281100
30,1.348500
40,0.992900
50,1.134400
60,1.096500
70,1.026200
80,0.888900
90,1.251800
100,1.021400


TrainOutput(global_step=354, training_loss=0.7043999123708004, metrics={'train_runtime': 1009.9443, 'train_samples_per_second': 5.644, 'train_steps_per_second': 0.351, 'total_flos': 2454630280667136.0, 'train_loss': 0.7043999123708004})

In [31]:
output_dir = '/content/drive/MyDrive/InkubaLM/outputs/'

In [32]:
# Save base model (optional, if not already public)
#model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/base_model"))

# Save LoRA adapter
model.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter_v2"))
tokenizer.save_pretrained(os.path.join(output_dir, "sft_model/balanced/lora_adapter_v2"))


('/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v2/tokenizer_config.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v2/special_tokens_map.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v2/tokenizer.model',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v2/added_tokens.json',
 '/content/drive/MyDrive/InkubaLM/outputs/sft_model/balanced/lora_adapter_v2/tokenizer.json')

In [38]:
v2_path = os.path.join(output_dir, "sft_model/balanced_supp/checkpoint-354")
os.makedirs(v2_path, exist_ok=True)
#inference_model = AutoModelForCausalLM.from_pretrained(ckpt_path,device_map="auto")
inference_model = model.eval()
results_df = multitask_v2.apply_inference_to_test_data(inference_model, tokenizer, test_dataset)


Generating Responses:   0%|          | 0/900 [00:00<?, ?it/s]

In [39]:
results_df[['ID', 'Response']].to_csv(os.path.join(v2_path,'submission_full_finetune_v2.csv'), index=False)
results_df[['ID', 'generated','Response']].head()

,ID,generated,Response
0,ID_f3c74c7b_sentiment_test__hausa,Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki Tsaka-tsaki,1
1,ID_aad19dbf_sentiment_test__hausa,Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa Kyakkyawa,0
2,ID_f6de0381_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2
3,ID_cbec84fe_sentiment_test__swahili,Chanya Chanya Chanya Chanya Chanya Chanya Chan...,0
4,ID_885caf5c_sentiment_test__hausa,Korau Korau Korau Korau Korau Korau Kora,2


In [40]:
submission_csv = results_df[['ID','Response']]

In [43]:
submission_csv.to_csv(os.path.join(v2_path,'submission_full_finetune_v2.csv'), index=False)

In [45]:
test_csv = pd.read_csv(os.path.join(v2_path,'submission_full_finetune_v2.csv'))

In [46]:
test_csv

,ID,Response
0,ID_f3c74c7b_sentiment_test__hausa,1
1,ID_aad19dbf_sentiment_test__hausa,0
2,ID_f6de0381_sentiment_test__hausa,2
3,ID_cbec84fe_sentiment_test__swahili,0
4,ID_885caf5c_sentiment_test__hausa,2
...,...,...
895,ID_88cd08fe_test__afrixnli_hau,1
896,ID_404bc9af_test__afrixnli_swa,1
897,ID_a399ed32_test__afrixnli_swa,1
898,ID_9f28a032_test__afrixnli_hau,1
